In [1]:
import gensim
import pandas as pd

In [2]:
df = pd.read_json('./Cell_Phones_and_Accessories_5.json', lines=True)

In [7]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [3]:
df.shape

(194439, 9)

In [4]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [5]:
#Simple Preprocessing and Tokenize

review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [6]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [11]:
#Initislizing the model


model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
    epochs=10
)

Build Vocab

In [13]:
model.build_vocab(review_text,progress_per=100)

Training

In [15]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(123013447, 167737950)

In [16]:
model.save('./saved_models/word2vec-amazon-cell-reviews.model')

In [17]:
model.wv.most_similar('bad')

[('terrible', 0.6695847511291504),
 ('shabby', 0.6576861143112183),
 ('good', 0.5935689210891724),
 ('horrible', 0.5906153917312622),
 ('awful', 0.5385581254959106),
 ('okay', 0.5274118781089783),
 ('crappy', 0.5258909463882446),
 ('legit', 0.5223528146743774),
 ('exciting', 0.5212365388870239),
 ('disappointing', 0.5153327584266663)]

In [19]:
#Store the words + embeddings
from gensim.models import KeyedVectors
word_vectors = model.wv
word_vectors.save('./saved_weights/word2vec-amazon-cell-reviews.wordvectors')

#load back
wv_my = KeyedVectors.load('./saved_weights/word2vec-amazon-cell-reviews.wordvectors', mmap='r')

In [60]:

wv_my['and']

array([-0.626155  , -0.90105426, -1.3408837 ,  0.20661478, -0.29260623,
        3.4004748 , -1.576609  , -0.18423922, -1.105542  , -0.5568614 ,
        1.3403294 ,  0.5320507 ,  1.9873713 ,  0.3990386 , -1.8265526 ,
        2.79107   , -0.64846534,  0.9956031 , -0.7182522 , -0.3051935 ,
        0.73368543,  0.3641481 , -0.33329108,  1.1841555 , -0.82690936,
        0.9897949 ,  1.4652265 , -0.1611386 ,  0.21398796,  0.02859775,
        3.1452181 , -1.9936785 , -0.20500019,  0.1837355 ,  1.138417  ,
       -0.7756844 ,  0.3818761 ,  2.7331164 ,  0.49901438,  2.249884  ,
       -0.70854044,  0.8645374 ,  2.7534196 , -1.3520013 ,  1.2688842 ,
       -0.34128428,  0.48041126,  0.02900871, -1.3170048 ,  0.01246584,
        0.11163396,  1.6502991 , -0.81986976, -2.7234879 , -0.8709188 ,
       -0.5363886 , -0.30064967,  0.41917357,  0.79155844,  1.3635225 ,
       -0.5161363 ,  2.4331474 ,  0.3764144 ,  0.9750039 , -2.0342782 ,
       -1.1691629 , -0.7347459 , -0.9768845 , -1.2816901 ,  0.16

In [50]:
wv_my.most_similar('king')

[('outdated', 0.5896090269088745),
 ('flagship', 0.5802364945411682),
 ('latest', 0.5730975270271301),
 ('world', 0.572501540184021),
 ('beast', 0.5722168684005737),
 ('symbian', 0.5707665085792542),
 ('newest', 0.5682141184806824),
 ('snapdragon', 0.5618101954460144),
 ('leaders', 0.5492086410522461),
 ('marketplace', 0.5434140563011169)]

In [49]:
#Save only by the key pair
model.wv.save_word2vec_format('./saved_weights/word2vec-amazon-cell-reviews.txt',binary=False,write_header=False)

In [68]:
import numpy as np
import tensorflow as tf
import os
from tensorboard.plugins import projector

model_vec = gensim.models.keyedvectors.KeyedVectors.load('./saved_weights/word2vec-amazon-cell-reviews.wordvectors')

max_size = len(model_vec.key_to_index)-1
w2v = np.zeros((max_size,model_vec.vector_size))

if not os.path.exists('projections'):
    os.makedirs('projections')

with open("projections/metadata.tsv", 'w+') as file_metadata:

    for i, word in enumerate(model_vec.index_to_key[:max_size]):

        #store the embeddings of the word
        w2v[i] = model_vec[word]

        #write the word to a file 
        file_metadata.write(word + '\n')

# sess = tf.InteractiveSession()
# with tf.device("/cpu:0"):
#     embedding = tf.Variable(w2v, trainable=False, name='embedding')
# tf.global_variables_initializer().run()
# saver = tf.train.Saver()
# writer = tf.summary.FileWriter('projections', sess.graph)
# config = projector.ProjectorConfig()
# embed= config.embeddings.add()
# embed.tensor_name = 'embedding'
# embed.metadata_path = 'metadata.tsv'
# projector.visualize_embeddings(writer, config)
# saver.save(sess, 'projections/model.ckpt', global_step=max_size)

# Create embedding variable
embedding = tf.Variable(w2v, dtype=tf.float32, trainable=False, name='embedding')

# Projector configuration
writer = tf.summary.create_file_writer('projections')
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(logdir='./projections/', config=config)

# Save checkpoint
saver = tf.train.Checkpoint(embedding=embedding)  # Assuming embedding is a variable
saver.save(file_prefix='projections/model.ckpt')

'projections/model.ckpt-1'

In [42]:
corpus = ["the", "king", "is", "wise", "and", "the", "queen", "is", "beautiful"]

# Prepare training data (center words and their corresponding context words)
center_word_data = []
context_word_data = []
window_size = 5

for sentence in corpus:
  for i in range(window_size, len(sentence) - window_size):  # Adjusted starting index
    center_word = sentence[i]
    context_words = sentence[i - window_size:i + window_size + 1]
    context_word_data.append(context_words[i])
    center_word_data.append(center_word)

'wise'